<h2 align = "center">Sprawozdanie mofit</h2>
<h4 align="center">Łukasz Puchała, Karol Grzywa</h4>

In [10]:
import numpy as np
from numba import njit
import math
import matplotlib.pyplot as plt
from math import sin, cos, pi, e, exp
from scipy.linalg import eigh
import Utilis
import plotly.subplots as sp
import plotly.express as px

m = 0.067 # m0
h = 1
at2meV = 27211.6
omega = 10/at2meV
ab = 0.05292
j = 0 + 1j
at2s = 2.42e-5*1e-12

L = 100/ab
N = 2
a = L/(2*N)

In [11]:
grid_N2_L100 = Utilis.ReadFile('Files/wezly_N_2_L_100.dat')


<h2 align = "center">generowanie macierzy lokalnych </h2>

In [12]:
grid_N2_L100 = Utilis.ReadFile('Files/wezly_N_2_L_100.dat')
wezly = np.array(grid_N2_L100)

In [13]:
nlg_N2_l100 = Utilis.ReadFile('Files/nlg_N_2_L_100.dat.txt')
nlg = np.array(nlg_N2_l100)

In [14]:
# funkcja, co bierze poz. wezla naroznego elementu k o loklanym nr i
print( Utilis.xy_nlg(1, 1, nlg, wezly) )
# wspóółrzędne rzeczywiste punktu 
print( 'wsp', Utilis.xryr(k= 1, ksi1= 0, ksi2= 0, nlg= nlg, wezly= wezly) )
j,i,s = Utilis.Sloc(a)
print(s)
print(j)
print(i)

(-50.0, -50.0)
wsp (-37.5, -37.5)
[ 3967.50807674  -991.87701918  -991.87701918   247.9692548
  1983.75403837  -495.93850959  1983.75403837  -495.93850959
   991.87701918  -991.87701918  3967.50807674   247.9692548
  -991.87701918  1983.75403837  1983.75403837  -495.93850959
  -495.93850959   991.87701918  -991.87701918   247.9692548
  3967.50807674  -991.87701918  -495.93850959  -495.93850959
  1983.75403837  1983.75403837   991.87701918   247.9692548
  -991.87701918  -991.87701918  3967.50807674  -495.93850959
  1983.75403837  -495.93850959  1983.75403837   991.87701918
  1983.75403837  1983.75403837  -495.93850959  -495.93850959
 15870.03230694   991.87701918   991.87701918 -3967.50807674
  7935.01615347  -495.93850959  1983.75403837  -495.93850959
  1983.75403837   991.87701918 15870.03230694 -3967.50807674
   991.87701918  7935.01615347  1983.75403837  -495.93850959
  1983.75403837  -495.93850959   991.87701918 -3967.50807674
 15870.03230694   991.87701918  7935.01615347  -495.938

<h2>Wyznaczanie macierzy lokalnych</h2>

In [15]:
Sgiven = Utilis.ReadFile("./Files/S_loc_N2_L100.dat.txt")
Tgiven = Utilis.ReadFile("./Files/T_loc_N2_L100.dat.txt")
V11given = Utilis.ReadFile("./Files/V_loc_N2_L100_element_11.dat.txt")
jt,it,T = Utilis.Tmatrix(m,0.0001)
js,Is,S = Utilis.Sloc(a)
jv,iv,V = Utilis.Vkmatrix(11,a,m,omega,nlg,wezly)
Tdif = T - np.array(Tgiven[2])
Sdiff = S - np.array(Sgiven[2])
Vdiff = V - np.array(V11given[2])


<h2>Rysunek odjętych od siebie macierzy wyznaczonych i danych</h2>

In [16]:
px.imshow(Tdif.reshape((9, 9)), x=np.unique(it), y=np.unique(jt), title='Tdif', color_continuous_scale='Viridis').show()
px.imshow(Sdiff.reshape((9, 9)), x=np.unique(Is), y=np.unique(js), title='Sdiff', color_continuous_scale='Plasma').show()
px.imshow(Vdiff.reshape((9, 9)), x=np.unique(iv), y=np.unique(jv), title='Vdiff', color_continuous_scale='Blues').show()

# Macierze globalne

In [17]:
Sinit, Hinit = Utilis.Gmatrix(N, nlg, wezly, a)
S, H = Utilis.Gboundary(N, Sinit, Hinit, nlg, wezly)

fig_S = px.imshow(S, title="S", color_continuous_scale= 'inferno')
fig_S.show()

fig_H = px.imshow(H, title="H", color_continuous_scale= 'viridis')
fig_H.show()

In [18]:
import pandas as pd
global_matrix_given = pd.read_csv("./Files/MacGlobSiH.dat", sep = r"\s+",header = None)
S_Given = np.array(global_matrix_given[2]).reshape(81,81)
H_Given = np.array(global_matrix_given[3]).reshape(81,81)

fig_S = px.imshow(S_Given-S, title="Sdiff", color_continuous_scale= 'inferno')
fig_S.show()

fig_H = px.imshow(H_Given-H, title="Hdiff", color_continuous_scale= 'viridis')
fig_H.show()


In [ ]:
from scipy.linalg import eigh
# solve generalized eigenproblem
E, C = eigh( H,S)
#psi = Utilis.CalculatePsi(C, 2,N,nlg,wezly)\
idx = 37
print(E[idx]*at2meV)
# reshape first eigenvector to 2D grid
#N = int(np.sqrt(C.shape[0]))  # 9 if 81
#psi = C[:, 0].reshape(N, N)

# plot real part
px.imshow(
    np.abs(C[idx]).reshape(9,9),
    color_continuous_scale="RdBu",
    title="Re(ψ₀)"
).show()

[-3.83683560e+07 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07
 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07
 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07
 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07
 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07
 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07
 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07
 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07 -3.83683560e+07
  1.08352852e+01  2.14336263e+01  2.14336263e+01  3.20319674e+01
  3.43116663e+01  3.43116664e+01  4.49100075e+01  4.49100075e+01
  4.98066898e+01  4.98066898e+01  5.77880475e+01  5.90938922e+01
  5.90938922e+01  6.04050309e+01  6.04050309e+01  6.96922333e+01
  6.96922333e+01  7.32830709e+01  7.32830709e+01  8.25702734e+01
  8.25702734e+01  8.58730449e+01  8.58730449e+01  8.87780943e+01
  8.89142113e+01  8.89142113e+01  9.64713860e+01  9.64713860e+01
  9.80652968e+01  9.80652